In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
train_models(env, models, episodes=50, steps=500, print_every=100)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: 0
----STEP 100 rewards----
Model 0: 0.7151839881512458
----STEP 200 rewards----
Model 0: 0.2105343690803636
----STEP 300 rewards----
Model 0: 0.7049240132771217
----STEP 400 rewards----
Model 0: 0.20037648774123795
------EPISODE 0 rewards------
Model 0: 1.8660329241550286
----STEP 0 rewards----
Model 0: 1
----STEP 100 rewards----
Model 0: 3.477669099783874
----STEP 200 rewards----
Model 0: 2.945530926997023
----STEP 300 rewards----
Model 0: 3.413401637540268
----STEP 400 rewards----
Model 0: 3.879705276964124
------EPISODE 1 rewards------
Model 0: 3.3489014684912135
----STEP 0 rewards----
Model 0: 0
----STEP 100 rewards----
Model 0: 0.6949009572641742
----STEP 200 rewards----
Model 0: 0.19045316776396248
----STEP 300 rewards----
Model 0: 0.8778223221640005
----STEP 400 rewards----
Model 0: 0.371554344115566
------EPISODE 2 rewards------
Model 0: 0.000162118154115376
----STEP 0 rewards----
Model 0: 1
----STEP 100 rewards----
Model 0: 2.4833369

----STEP 200 rewards----
Model 0: 8.075261144115501
----STEP 300 rewards----
Model 0: 9.48272776806251
----STEP 400 rewards----
Model 0: 8.890835290329171
------EPISODE 27 rewards------
Model 0: 10.303585722700504
----STEP 0 rewards----
Model 0: 2
----STEP 100 rewards----
Model 0: 6.450183527237755
----STEP 200 rewards----
Model 0: 8.869830168758341
----STEP 300 rewards----
Model 0: 9.281539425153845
----STEP 400 rewards----
Model 0: 8.691648904480068
------EPISODE 28 rewards------
Model 0: 9.112739738504523
----STEP 0 rewards----
Model 0: 1
----STEP 100 rewards----
Model 0: 2.477870151262202
----STEP 200 rewards----
Model 0: 2.9554806491888286
----STEP 300 rewards----
Model 0: 4.41558369655808
----STEP 400 rewards----
Model 0: 5.856985549026319
------EPISODE 29 rewards------
Model 0: 6.3029092959058275
----STEP 0 rewards----
Model 0: 1
----STEP 100 rewards----
Model 0: 3.487321859007359
----STEP 200 rewards----
Model 0: 3.9504979697190947
----STEP 300 rewards----
Model 0: 4.4128551169

In [ ]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_1")